# Real-word Trade-in Values of Car Brands

## Import packages

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict

In [2]:
# This code shows how to scrape twitter by using the snscrape python API
# Please make sure the snscrape has been installed in the local environment
# The snscape package can be installed by: pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git
# The Python version has to be greater than 3.8
# An alternative Python method is to to execute CLI commands in Python. 
# Please check this link for more details: https://colab.research.google.com/drive/1ugr1biGxV9C2OwzS3HEh3KM0z2Xg44jb?usp=sharing

import snscrape.modules.twitter as sntwitter
import pandas as pd

key_word = "#ChinaVirus"  # Declare the key word used to search tweets
from_date = "2020-03-10" # Declare a start date
end_date = '2020-03-17'  # Declare a end date
count = 500          # The maximum number of tweets
tweets_list_keyword = [] # A list used to store the returned results for keyword search
tweets_list_user = []    # A list used to store the retuned results for user search


#### Scraping tweets from a text search query ####
command_keyword = key_word+' since:'+from_date+' until:'+end_date # Define a string command for Scraper Api
print("Scraping data for keyword:",key_word)
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(command_keyword).get_items()):
    tweets_list_keyword.append([tweet.id, tweet.content, tweet.user.username, tweet.user.followersCount,tweet.user.listedCount,tweet.retweetedTweet]) # Append returned results to list
    if i>count:
        break;
#Create a dataframe from the tweets list above 
tweets_df_keyword = pd.DataFrame(tweets_list_keyword, columns=['Tweet Id', 'Text', 'Username', 'followers','listedcount','retweeted'])
tweets_df_keyword.to_csv("before_318.csv",index=False) # Export to a csv file
print("Scraped data have been exported to the csv file")

Scraping data for keyword: #ChinaVirus
Scraped data have been exported to the csv file


In [ ]:
key_word = "#ChinaVirus"  # Declare the key word used to search tweets
from_date = "2020-03-18" # Declare a start date
end_date = '2020-03-24'  # Declare a end date
count = 500          # The maximum number of tweets
tweets_list_keyword = [] # A list used to store the returned results for keyword search
tweets_list_user = []    # A list used to store the retuned results for user search


#### Scraping tweets from a text search query ####
command_keyword = key_word+' since:'+from_date+' until:'+end_date # Define a string command for Scraper Api
print("Scraping data for keyword:",key_word)
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(command_keyword).get_items()):
    tweets_list_keyword.append([tweet.id, tweet.content, tweet.user.username, tweet.user.followersCount,tweet.user.listedCount,tweet.retweetedTweet]) # Append returned results to list
    if i>count:
        break;
#Create a dataframe from the tweets list above 
tweets_df_keyword = pd.DataFrame(tweets_list_keyword, columns=['Tweet Id', 'Text', 'Username', 'followers','listedcount','retweeted'])
tweets_df_keyword.to_csv("after_318.csv",index=False) # Export to a csv file
print("Scraped data have been exported to the csv file")

Scraping data for keyword: #ChinaVirus


In [ ]:
tweets_df_keyword

In [ ]:

from pathlib import Path
path = Path.cwd() # current working directory


In [ ]:
import re
text = tweets_df_keyword.Text.to_list()
non_tweet_users = []
for t in text:
    
    non_tweet_users.append(re.findall(r'[#]\S*', t))


In [ ]:
non_tweet_users

## Task: Scrapping data and process the data using NLTK 

### Scrapping latest 5000 posts and store in post_text

In [ ]:
post_text=[]
for i in range(994,1095):
    url='https://forums.edmunds.com/discussion/1212/general/x/real-world-trade-in-values/p'+str(i)
    page=requests.get(url)
    page_text=page.text
    soup=BeautifulSoup(page_text, 'html.parser')
    post_text_elem= soup.find_all(class_="Message userContent")
    for item in post_text_elem:
        post_text.append(item.text.lstrip())
post_text = post_text[35:5035]


### Tokenize/remove stopwords/lemmatize

In [ ]:
def tokenizer(words) -> list:
    tokenizer = RegexpTokenizer(r'[\w-]+')
    tokens = tokenizer.tokenize(words.lower())

    stopwords_set = set(stopwords.words('english')) # use build in english package
    lemmatizer = WordNetLemmatizer()

    return [lemmatizer.lemmatize(token, pos='v') for token in tokens if token not in stopwords_set]

#apply tokenizer function to each post, using set to remove the repeat and store in processed text
processed_text = []
for comment in post_text:
    processed_text.append(list(set(tokenizer(comment))))

## Task: Find the top 10 brands from frequency counts

$\textbf{Note:}$ Some people just use model name of a car instead of mentioning the brand name. Thus, we choose to convert the model name to the brand name and count frequency of each brand based on the converted text.

To do so, we created a model_csv that stores the informaiton of car brand and the popular models under the brands.

### Import model_csv and process

In [ ]:
model_grid = pd.read_csv(path.joinpath('models.csv'))

model_grid['Brands'] = model_grid['Brands'].str.lower()
model_grid['model']  = model_grid['model'].str.lower()
model_grid = model_grid[['Brands','model']]

In [ ]:
model_grid

### Calculate model frequency  

In [ ]:
# Replace the model with its brand in the processed_text
remove_dup = []
for comment in processed_text: 
    for i in range(len(comment)): 
        if comment[i] in model_grid.model.tolist():
            condition = model_grid['model'] == comment[i]
            brand_index = model_grid.index[condition].tolist()
            comment[i] = model_grid['Brands'].iloc[brand_index[0]]
    remove_dup.append(list(set(comment)))
fre = defaultdict(int)

In [ ]:
for p in remove_dup:
    for w in p:
        if w in model_grid.Brands.tolist():
            fre[w] += 1

### Convert the brand frequency to dataframe and keep the top 10 brands

In [ ]:
brand_frequency = dict(fre)
br = brand_frequency.keys()
fre = brand_frequency.values()
df = pd.DataFrame()
df['brand'] = br
df['frequency'] = fre

#include top 10 brand and its frequency
top = df.sort_values(by = 'frequency', ascending= False).head(10)

#top 10 brand
top

In [ ]:
top_10 = top.brand.tolist() ## create list that record the name of top 10 brand.

## Task: Calculate lift ratios for associations between top 10 brands

### Getting the combination of 10 brands to calculate the lift

In [ ]:
from itertools import combinations
comb = list(combinations(top_10,2))

### Create a sub-dataframe to store the model for top10 brands

In [ ]:
target_df = model_grid[model_grid['Brands'].isin(top_10)]
target_df = target_df.reset_index(drop = True)
target_df

### Replace the model with its brand in the processed_text

In [ ]:
for comment in processed_text: 
    for i in range(len(comment)): 
        if comment[i] in target_df.model.tolist():
            condition = target_df['model'] == comment[i]
            brand_index = target_df.index[condition].tolist()
            comment[i] = target_df['Brands'].iloc[brand_index[0]]

### Getting a narrowed list of the posts that contains target brands

In [ ]:
target_comment = []
for comment in processed_text:
    for words in comment:
        if words in top_10:
            target_comment.append(list(set(comment)))
#remove duplicate
target_comment_distinct = []
for i in target_comment:
    if i not in target_comment_distinct:
        target_comment_distinct.append(i)
        


### Calculate the #(A,B) 

In [ ]:
pair_frequency = {}
for i in comb:
    pair_frequency[i] = 0
for pair in comb:
    for comm in target_comment_distinct:
        if pair[0] in comm and pair[1] in comm:
            pair_frequency[pair] +=1

In [ ]:
pair_frequency

### Calculate lift

In [ ]:
lift = defaultdict(int)
for i in pair_frequency.keys():
    lift[i] = (5000*pair_frequency[i]) / (brand_frequency[i[0]]*brand_frequency[i[1]])

In [ ]:
for i in lift:
    if lift[i] == 0: ## some time two brand has not been mentioned together at all,we need to set a base value for lift
        lift[i] = 1

In [ ]:
df_lifts = pd.DataFrame(columns=top_10,index=top_10)
for brand in top_10: 
    df_lifts[brand][brand] = '-'
for brands in lift:
    a,b = brands
    df_lifts[a][b] = (lift[brands])
    df_lifts[b][a] = '-'

In [ ]:
df_lifts

In [ ]:
top_brand_lifts = pd.DataFrame(columns=top_10,index=top_10)
for brands in lift:
    a,b = brands
    top_brand_lifts[a][b] = (1/lift[brands])
    top_brand_lifts[b][a] = (1/lift[brands])
for brand in top_10: 
    top_brand_lifts[brand][brand] = 0

In [ ]:
top_brand_lifts

## Task:  Plot the top 10 brands on a multidimensional scaling (MDS) map

In [ ]:
from sklearn import manifold
import matplotlib.pyplot as plt
mds = manifold.MDS(dissimilarity='euclidean',random_state=522)
mds_fit = mds.fit(top_brand_lifts)
coords = mds.fit_transform(top_brand_lifts)

In [ ]:
x,y = coords.T

plt.figure(figsize=(10, 10))
plt.scatter(x,y)

for i, txt in enumerate(top_10):
    plt.annotate(txt, (x[i], y[i]), xycoords='data',
             xytext=(20, -20), textcoords='offset points',
             size=13, ha='right', va="center",
             bbox=dict(boxstyle="round", alpha=0.1),
             arrowprops=dict(arrowstyle="wedge,tail_width=0.5", alpha=0.1))
    
plt.yticks([])
plt.xticks([])
plt.title('Multidimensional Scaling: Top 10 Brands \n (Euclidean Distances = 1 / lift)')

 ## Task: Offer insights to two top 10 brands based on the analysis above.

Based on our analysis of the real-trade-in value forum on Edmunds.com, Volkswagen and Chevrolet appear to have the highest lift (3.25), which means that there’s a positive association between these two brands. Chevrolet is more likely to be mentioned in conjunction with Volkswagen than other brands. The frequency of occurrence of Chevrolet and Volkswagen is 40 among the total comments scraped.

One of the possible reasons for such a high co-occurrence of these 2 brands in comments is that they have either very similar or very different trade-in values. When people discuss trade-in values, they tend to mention both of these brands at the same time to make comparisons. Therefore, brand managers of both Volkswagen and Chevrolet should be aware that the other brand is a potential competitor in the used car market. Some buyers who want to change hands their cars shortly might consider the trade-in values when buying the new car. Potential adjustments in production or after-sales service may affect the trade-in value, thus affecting the sales and profits.

According to our research of the two brands, we found that they have overlapped target markets. Although these two brands may have different strategies in the new car market, when it comes to used car sales, customers’ standards tend to become similar. And sometimes, the trade-in value would also become an important feature. Especially for target customers of Volkswagen and Chevrolet, the age range indicates they may be in a stage that is not that stable and face the trade-in in the near future such as single to married or married to have children. Therefore, it is worth thinking for the brand managers of Volkswagen and Chevrolet that the higher trade-in value/reliability can become a selling point for those customers.


## Task: Analyze most frequently mentioned attributes of cars in the discussions

To calculate the most frequently mentioned attributes of cars in the discussions, we then look for the attributes from the narrowed list of the posts that contains brand (the target comment we got in previous section).

But rather than using this list directly, we remove the verbs, conjunctions and prepositions using the tagging feature of NLTK.

### Tagging words in target comment

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
wordlist = []
for i in range(len(target_comment)):
    for j in range(len(target_comment[i])):
        wordlist.append(target_comment[i][j])
text = ' '.join(wordlist).lower()

In [ ]:
tokens = nltk.word_tokenize(text)

In [ ]:
tags = nltk.pos_tag(tokens)

In [ ]:
## list of noun and adj words
n_adj_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' 
                                             or pos == 'VBD' or pos == 'JJ' or pos == 'JJR' or pos == 'JJS'
                                             or pos =='RB' or pos =='RBR'or pos =='RBS')]
## list of adj only
adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]

Firstly we did a word cloud based on the noun and adjs in the selecteed posts, to have a brief idea of the attributes that have involved in the discussion.

In [ ]:
n_adj_string=(" ").join(n_adj_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

### Calculate the frequency of words 

In [ ]:
#count the words frequency and sort
from collections import Counter
counts = Counter(n_adj_list)


## Task: Create the list of attribute based on the frequently mentioned words

After counting the occurrences of each selected word, ranked them by the number of occurrences. We manually selected meaningful descriptive words for cars and categorized them into 11 attributes: range, performance, comfort, vehicle architecture, safety, drivability, interior, sound, thermal management, energy management, appearance, and cost efficiency. (Reference of the attribute and how they have been categorized:https://www.avl.com/-/vehicle-attributes)

Since we have already counted the occurrences of each particular word that describes vehicles, we summed up the number of occurrences of each word under each of the attribute categories to get the frequency of each attribute.

The result was inputed into a attribute.csv.

In [ ]:
#import attributes csv
attributes = pd.read_csv(path.joinpath('attributes.csv'))

In [ ]:
attributes.head()

## Task: Find attributes that are most strongly associated with top 5 brands

In [ ]:
s = attributes[['range_f', 'performance_f', 'comfort_f', 'vehicle_architecture_f','safety_f', 'drivability_f', 'interior_sound_f', 'thermal_management_f', 'energy_management_f', 'appearance_f','cost_efficiency_f']].sum()

In [ ]:

attr = s.sort_values(ascending=False)
attr = pd.DataFrame({'attributes':attr.index, 'frequency':attr.values})
attr

### Top 5 car brands' frequency and lift for attributes that mentioned in the discussion

In [ ]:
##From TASK A we get the most frequently mentioned brands are:
top5 = df.sort_values(by = 'frequency', ascending= False).head(5)
top5

In [ ]:
import itertools
attlist = []
brandlist = []
for i in attr['attributes']:
    attlist.append(i)
    
for i in top5['brand']:
    brandlist.append(i) 

all_combinations = list(itertools.product(brandlist, attlist))


In [ ]:
com = []
for i in range(len(all_combinations)):
    com.append(all_combinations[i])

#to remove duplicated combinations
com = list(set(com))

## Task: Calculate the lift values for each pair of (brand, attribute)

### Calculate the frequency #(Brand, attribute)

In [ ]:
import copy
comment_copy = copy.deepcopy(target_comment)

In [ ]:
#to replace each specific attribute words with attributes categories in the text

for j in range(len(target_comment)):
    for i in range(len(target_comment[j])):
        if target_comment[j][i] in attributes.cost_efficiency.dropna().tolist():
            target_comment[j][i] = 'cost_efficiency_f'
            continue
        if target_comment[j][i] in attributes.appearance.dropna().tolist():
            target_comment[j][i] = 'appearance_f'
            continue
        if target_comment[j][i] in attributes.drivability.dropna().tolist():
            target_comment[j][i] = 'drivability_f'
            continue
        if target_comment[j][i] in attributes.range.dropna().tolist():
            target_comment[j][i] = 'range_f'
            continue
        if target_comment[j][i] in attributes.vehicle_architecture.dropna().tolist():
            target_comment[j][i] = 'vehicle_architecture_f'
            continue
        if target_comment[j][i] in attributes.performance.dropna().tolist():
            target_comment[j][i] = 'performance_f'
            continue
        if target_comment[j][i] in attributes.comfort.dropna().tolist():
            target_comment[j][i] = 'comfort_f'
            continue
        if target_comment[j][i] in attributes.safety.dropna().tolist():
            target_comment[j][i] = 'safety_f'
            continue
        if target_comment[j][i] in attributes['interior_sound'].tolist():
            target_comment[j][i] = 'interior_sound_f'
            continue
        if target_comment[j][i] in attributes.thermal_management.dropna().tolist():
            target_comment[j][i] = 'thermal_managemen_f'
            continue
        if target_comment[j][i] in attributes.energy_management.dropna().tolist():
            target_comment[j][i] = 'energy_management_f'
            continue

In [ ]:
## Calculate the frequency #(Brand, attribute)
pair_f = defaultdict(int)
for pair in com:

    for comment in target_comment:
        if pair[0] in comment and pair[1] in comment:

            pair_f[pair] +=1


In [ ]:
## Store into a dictionary
brand_dict = {}
for (key, value) in brand_frequency.items():
    if key in top5.brand.tolist():
        brand_dict[key] = value
att_dict = dict(zip(attr.attributes, attr.frequency))
att_dict

In [ ]:
#Calculate Lift
liftC = defaultdict(int)
for i in pair_f.keys():
    
    liftC[i] = (5000*pair_f[i]) / (brand_dict[i[0]]*att_dict[i[1]])

### Create dataframe that store the lift values for each brand

In [ ]:
honda_attr_lift = pd.DataFrame(columns=["attribute","Lift"])
for i in liftC.keys():
    if i[0] == 'honda':
        honda_attr_lift = honda_attr_lift.append({'attribute': i[1], 'Lift': liftC[i]}, ignore_index=True)

In [ ]:
honda_attr_lift = honda_attr_lift.sort_values(by = 'Lift', ascending= False)

In [ ]:
honda_attr_lift

In [ ]:
chevrolet_attr_lift = pd.DataFrame(columns=["attribute","Lift"])
for i in liftC.keys():
    if i[0] == 'chevrolet':
        chevrolet_attr_lift = chevrolet_attr_lift.append({'attribute': i[1], 'Lift': liftC[i]}, ignore_index=True)

chevrolet_attr_lift = chevrolet_attr_lift.sort_values(by = 'Lift', ascending= False)

In [ ]:
chevrolet_attr_lift

In [ ]:
toyota_attr_lift = pd.DataFrame(columns=["attribute","Lift"])
for i in liftC.keys():
    if i[0] == 'toyota':
        toyota_attr_lift = toyota_attr_lift.append({'attribute': i[1], 'Lift': liftC[i]}, ignore_index=True)

toyota_attr_lift = toyota_attr_lift.sort_values(by = 'Lift', ascending= False)

In [ ]:
toyota_attr_lift

In [ ]:
ford_attr_lift = pd.DataFrame(columns=["attribute","Lift"])
for i in liftC.keys():
    if i[0] == 'ford':
        ford_attr_lift = ford_attr_lift.append({'attribute': i[1], 'Lift': liftC[i]}, ignore_index=True)

ford_attr_lift = ford_attr_lift.sort_values(by = 'Lift', ascending= False)

In [ ]:
ford_attr_lift

In [ ]:
fiat_attr_lift = pd.DataFrame(columns=["attribute","Lift"])
for i in liftC.keys():
    if i[0] == 'fiat':
        fiat_attr_lift = fiat_attr_lift.append({'attribute': i[1], 'Lift': liftC[i]}, ignore_index=True)

fiat_attr_lift = fiat_attr_lift.sort_values(by = 'Lift', ascending= False)

In [ ]:
fiat_attr_lift

## Task: Analyze attribute detail of top 5 brand and provide insights to product manager, and marketing/advertising team of the brands.

First import detail_attribute dataset. This is just a csv with a list of uncateforized attribute.

In [ ]:
detial_attributes = pd.read_csv(path.joinpath('detail_attribute.csv'))

In [ ]:
detial_attributes = detial_attributes.detail_attribute.values

### For Honda:

In [ ]:
## filter out honda related comment
honda_comment = []
for comment in comment_copy:
    if 'honda' in comment:
        honda_comment.append(list(set(comment)))


In [ ]:
## calculate the frequency of each detailed attribute
honda_attr = defaultdict(int)
for comment in honda_comment:
    for attr in detial_attributes:
        if attr in comment:

            honda_attr[attr] +=1


In [ ]:
honda_attr = pd.DataFrame(list(zip(honda_attr.keys(), honda_attr.values())),columns =['attribute', 'frequency'])
honda_attr = honda_attr.sort_values(by = 'frequency', ascending= False)

In [ ]:
honda_attr.head(10)

#### Advice for product managers:

The two top 5 brands that are highly associated with Honda are Fiat and Ford. Compared to Fiat and Ford, Honda has a relatively low lift score in the energy management and performance attributes. In order to become more competitive, Honda product managers should consider enhancing the performance and energy management aspects. 

Specifically, the Wordcloud (see below) for energy management-related comment contains the phrase “auxiliary low”. Thus, product managers should focus on fixing the problem on auxiliary of Honda’s products.

#### Honda performance related comment visualization:

In [ ]:
honda_performance_comment = []
for j in range(len(honda_comment)):
    for i in range(len(honda_comment[j])):
        if honda_comment[j][i] in attributes.performance.dropna().tolist():
            honda_performance_comment.append(list(set(honda_comment[j])))

In [ ]:
honda_performance_list = []
for i in range(len(honda_performance_comment)):
    for j in range(len(honda_performance_comment[i])):
        honda_performance_list.append(honda_performance_comment[i][j])
text = ' '.join(honda_performance_list).lower()
tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Honda energy_management related comment visualization:

In [ ]:
honda_energy_management_comment = []
for j in range(len(honda_comment)):
    for i in range(len(honda_comment[j])):
        if honda_comment[j][i] in attributes.energy_management.dropna().tolist():
            honda_energy_management_comment.append(list(set(honda_comment[j])))

In [ ]:
honda_energy_management_list = []
for i in range(len(honda_energy_management_comment)):
    for j in range(len(honda_energy_management_comment[i])):
        honda_energy_management_list.append(honda_energy_management_comment[i][j])
text = ' '.join(honda_energy_management_list).lower()
tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Advices for marketing/advertising managers:

One of the most important features of Honda is that the brand is considered to be very cost-efficient. The lift score for the cost-efficient attribute is the highest among all the top 5 brands. Since this is a forum on trade-in values, we can consider Honda to have a relatively high trade-in value among the top 5 brands. The marketing campaign can focus on this attribute. Honda is also strongly associated with the attributes: comfort and interior and sound. However, as discussed above it is considered to be a low-end brand. Thus with a small budget, the buyer can get a really comfortable car with a nice interior. This proves that buying a Honda car is really a good deal. Thus, we believe the advertisement of Honda should definitely contain these key ideas.

#### Honda comfort related comment visualization:

In [ ]:
honda_comfort_comment = []
for j in range(len(honda_comment)):
    for i in range(len(honda_comment[j])):
        if honda_comment[j][i] in attributes.comfort.dropna().tolist():
            honda_comfort_comment.append(list(set(honda_comment[j])))

In [ ]:
honda_comfort_list = []
for i in range(len(honda_comfort_comment)):
    for j in range(len(honda_comfort_comment[i])):
        honda_comfort_list.append(honda_comfort_comment[i][j])
text = ' '.join(honda_comfort_list).lower()
tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

### For Chevrolet

In [ ]:
chevrolet_comment = []
for comment in comment_copy:
    if 'chevrolet' in comment:
        chevrolet_comment.append(list(set(comment)))

In [ ]:
chevrolet_attr = defaultdict(int)
for comment in chevrolet_comment:
    for attr in detial_attributes:
        if attr in comment:

            chevrolet_attr[attr] +=1
chevrolet_attr = pd.DataFrame(list(zip(chevrolet_attr.keys(), chevrolet_attr.values())),columns =['attribute', 'frequency'])
chevrolet_attr = chevrolet_attr.sort_values(by = 'frequency', ascending= False)

In [ ]:
chevrolet_attr.head(10)

#### Advices for product managers:

The brands that are highly associated with Chevrolet are Toyota and Ford. Compared to them, Chevrolet is relatively weak in interior and safety. Thus the product managers of Chevrolet should focus on enhancing the brand’s interior design and safety. One of the words that commonly appear in the Chevrolet Wordcloud is “frumpy”. People are considered Chevrolet to be old fashion. Thus product manager might need to focus more on the design of the car.

#### Chevrolet safety related comment visualization:

In [ ]:
chevrolet_safety_comment = []
for j in range(len(chevrolet_comment)):
    for i in range(len(chevrolet_comment[j])):
        if chevrolet_comment[j][i] in attributes.safety.dropna().tolist():
            chevrolet_safety_comment.append(list(set(chevrolet_comment[j])))

In [ ]:
chevrolet_safety_list = []
for i in range(len(chevrolet_safety_comment)):
    for j in range(len(chevrolet_safety_comment[i])):
        chevrolet_safety_list.append(chevrolet_safety_comment[i][j])
text = ' '.join(chevrolet_safety_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Chevrolet interial related comment visualization:

In [ ]:
chevrolet_interior_sound_comment = []
for j in range(len(chevrolet_comment)):
    for i in range(len(chevrolet_comment[j])):
        if chevrolet_comment[j][i] in attributes.interior_sound.dropna().tolist():
            chevrolet_interior_sound_comment.append(list(set(chevrolet_comment[j])))

In [ ]:
chevrolet_interior_sound_list = []
for i in range(len(chevrolet_interior_sound_comment)):
    for j in range(len(chevrolet_interior_sound_comment[i])):
        chevrolet_interior_sound_list.append(chevrolet_interior_sound_comment[i][j])
text = ' '.join(chevrolet_interior_sound_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

####  Advices for marketing/advertising managers:

Like Honda, Chevrolet is also considered to be cost-efficient. The phrase “good resell” has appeared in Wordcloud several times. Thus, the marketing team should definitely focus on the resell value of Chevrolet. Notably, in the performance Wordcloud, the word “snow” appears multiple times. Chevrolet performs well in snow weather. Thus the marketing team should consider promoting more in the snowy region such as Michigan and Montreal.

#### Chevrolet performance related comment visualization:

In [ ]:
chevrolet_performance_comment = []
for j in range(len(chevrolet_comment)):
    for i in range(len(chevrolet_comment[j])):
        if chevrolet_comment[j][i] in attributes.performance.dropna().tolist():
            chevrolet_performance_comment.append(list(set(chevrolet_comment[j])))

In [ ]:
chevrolet_performance_list = []
for i in range(len(chevrolet_performance_comment)):
    for j in range(len(chevrolet_performance_comment[i])):
        chevrolet_performance_list.append(chevrolet_performance_comment[i][j])
text = ' '.join(chevrolet_performance_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

### For Toyota

In [ ]:
toyota_comment = []
for comment in comment_copy:
    if 'toyota' in comment:
        toyota_comment.append(list(set(comment)))

In [ ]:
toyota_attr = defaultdict(int)
for comment in toyota_comment:
    for attr in detial_attributes:
        if attr in comment:

            toyota_attr[attr] +=1
toyota_attr = pd.DataFrame(list(zip(toyota_attr.keys(), toyota_attr.values())),columns =['attribute', 'frequency'])
toyota_attr = toyota_attr.sort_values(by = 'frequency', ascending= False)

In [ ]:
toyota_attr.head(10)

#### Advice for product managers:

Toyota is strongly associated with Chevrolet and Fiat. Compared to these two brands, Toyota does not have a strong performance and energy management. Closely examining the Wordclouds generated based on the performance and energy management comment, we found the word “engine” keeps showing up. Also, people mentioned GMC Denali, a very high-performance truck, while discussing Toyota’s energy management. These suggest that Toyota’s product managers really should focus on developing products that can provide better performance and energy management. Moreover, like Honda, Toyota is considered to be a low-end brand. For possible market expansion, the product managers might explore the possibility of developing more high-end models.

#### Toyota performance related comment visualization:

In [ ]:
toyota_performance_comment = []
for j in range(len(toyota_comment)):
    for i in range(len(toyota_comment[j])):
        if toyota_comment[j][i] in attributes.performance.dropna().tolist():
            toyota_performance_comment.append(list(set(toyota_comment[j])))

In [ ]:
toyota_performance_list = []
for i in range(len(toyota_performance_comment)):
    for j in range(len(toyota_performance_comment[i])):
        toyota_performance_list.append(toyota_performance_comment[i][j])
text = ' '.join(toyota_performance_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Toyota energy_management related comment visualization:

In [ ]:
toyota_energy_management_comment = []
for j in range(len(toyota_comment)):
    for i in range(len(toyota_comment[j])):
        if toyota_comment[j][i] in attributes.energy_management.dropna().tolist():
            toyota_energy_management_comment.append(list(set(toyota_comment[j])))

In [ ]:
toyota_energy_management_list = []
for i in range(len(toyota_energy_management_comment)):
    for j in range(len(toyota_energy_management_comment[i])):
        toyota_energy_management_list.append(toyota_energy_management_comment[i][j])
text = ' '.join(toyota_energy_management_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Advices for marketing/advertising managers:

One of the attributes that are strongly associated with Toyota is comfort. Based on the Wordcloud analysis, people mentioned drive space and big navigator. People consider it to be low-end but fancy enough. Thus we consider Toyota to be a family-friendly car brand because with a limited budget one can get a fancy enough and spacious car. The marketing direction should focus on building a family-friendly image.

#### Toyota comfort related comment visualization:

In [ ]:
toyota_comfort_comment = []
for j in range(len(toyota_comment)):
    for i in range(len(toyota_comment[j])):
        if toyota_comment[j][i] in attributes.comfort.dropna().tolist():
            toyota_comfort_comment.append(list(set(toyota_comment[j])))

In [ ]:
toyota_comfort_list = []
for i in range(len(toyota_comfort_comment)):
    for j in range(len(toyota_comfort_comment[i])):
        toyota_comfort_list.append(toyota_comfort_comment[i][j])
text = ' '.join(toyota_comfort_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

### For Ford

In [ ]:
ford_comment = []
for comment in comment_copy:
    if 'ford' in comment:
        ford_comment.append(list(set(comment)))

In [ ]:
ford_attr = defaultdict(int)
for comment in ford_comment:
    for attr in detial_attributes:
        if attr in comment:

            ford_attr[attr] +=1
ford_attr = pd.DataFrame(list(zip(ford_attr.keys(), ford_attr.values())),columns =['attribute', 'frequency'])
ford_attr = ford_attr.sort_values(by = 'frequency', ascending= False)

In [ ]:
ford_attr.head(10)

#### Advices for product managers:
The brands that are highly associated with Ford are Volkswagen and Subaru. Compared with other brands, Ford is considered to be safer. Thus the product manager should realize this strength and keep the current advantage. However, the product manager should also pay attention to the appearance of the vehicles. According to the text visualization, some words such as ‘frumpy’ and ‘look’ appear frequently, showing that the interior design could be adjusted to further improve the quality.


#### Ford safety related comment visualization:

In [ ]:
ford_safety_comment = []
for j in range(len(ford_comment)):
    for i in range(len(ford_comment[j])):
        if ford_comment[j][i] in attributes.safety.dropna().tolist():
            ford_safety_comment.append(list(set(ford_comment[j])))

In [ ]:
ford_safety_list = []
for i in range(len(ford_safety_comment)):
    for j in range(len(ford_safety_comment[i])):
        ford_safety_list.append(ford_safety_comment[i][j])
text = ' '.join(ford_safety_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Ford appearance related comment visualization:

In [ ]:
ford_appearance_comment = []
for j in range(len(ford_comment)):
    for i in range(len(ford_comment[j])):
        if ford_comment[j][i] in attributes.appearance.dropna().tolist():
            ford_appearance_comment.append(list(set(ford_comment[j])))

In [ ]:
ford_appearance_list = []
for i in range(len(ford_appearance_comment)):
    for j in range(len(ford_appearance_comment[i])):
        ford_appearance_list.append(ford_appearance_comment[i][j])
text = ' '.join(ford_appearance_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Advices for marketing/advertising managers:

Similar to Honda and Chevrolet, Ford is also considered to be cost-efficient. According to Wordcloud, people usually describe Ford to be a good deal, and suitable for ‘commuting’. Thus, the marketing team should potentially adjust the market positioning strategy accordingly and focus more on those people with higher demand on daily commuting. In addition, Ford brings comfort to drivers, showing it is also suitable for short business trips by car. Thus, the market team should consider promoting more advertisements in the business district.

#### Ford performance related comment visualization:

In [ ]:
ford_peformance_comment = []
for j in range(len(ford_comment)):
    for i in range(len(ford_comment[j])):
        if ford_comment[j][i] in attributes.appearance.dropna().tolist():
            ford_peformance_comment.append(list(set(ford_comment[j])))

In [ ]:
ford_performance_list = []
for i in range(len(ford_peformance_comment)):
    for j in range(len(ford_peformance_comment[i])):
        ford_performance_list.append(ford_peformance_comment[i][j])
text = ' '.join(ford_performance_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

### For Fiat:

In [ ]:
fiat_comment = []
for comment in comment_copy:
    if 'fiat' in comment:
        fiat_comment.append(list(set(comment)))

In [ ]:
fiat_attr = defaultdict(int)
for comment in fiat_comment:
    for attr in detial_attributes:
        if attr in comment:

            fiat_attr[attr] +=1
fiat_attr = pd.DataFrame(list(zip(fiat_attr.keys(), fiat_attr.values())),columns =['attribute', 'frequency'])
fiat_attr = fiat_attr.sort_values(by = 'frequency', ascending= False)

In [ ]:
fiat_attr.head(10)

#### Advices for product managers:

The brands that are highly associated with Fiat are Nissan and BMW. But by the text visualization, people mention ‘damage’ and ‘accident’ frequently. So Fiat’s product manager should definitely focus on improving driving safety, and the tire value, in particular. In addition, customers seem to appreciate those electrical vehicles more, compared with those ones involving traditional gas for energy. In this case, the product manager should also concentrate on the product transformation, and produce more vehicles popular among customers.

#### Fiat safety related comment visualization:

In [ ]:
fiat_safety_comment = []
for j in range(len(fiat_comment)):
    for i in range(len(fiat_comment[j])):
        if fiat_comment[j][i] in attributes.safety.dropna().tolist():
            fiat_safety_comment.append(list(set(fiat_comment[j])))

In [ ]:
fiat_safety_list = []
for i in range(len(fiat_safety_comment)):
    for j in range(len(fiat_safety_comment[i])):
        fiat_safety_list.append(fiat_safety_comment[i][j])
text = ' '.join(fiat_safety_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Fiat energy_management related comment visualization:

In [ ]:
fiat_energy_management_comment = []
for j in range(len(fiat_comment)):
    for i in range(len(fiat_comment[j])):
        if fiat_comment[j][i] in attributes.energy_management.dropna().tolist():
            fiat_energy_management_comment.append(list(set(fiat_comment[j])))

In [ ]:
fiat_energy_management_list = []
for i in range(len(fiat_energy_management_comment)):
    for j in range(len(fiat_energy_management_comment[i])):
        fiat_energy_management_list.append(fiat_energy_management_comment[i][j])
text = ' '.join(fiat_energy_management_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Advices for marketing/advertising managers:

According to Wordcloud, customers value the vehicle's appearance a lot by frequently mentioning the pretty colors and the right size for the garage. Thus, the marketing manager should emphasize the well-designed outlook and its convenience for crowded places. Moreover, similar to Chevrolet, Fiat is considered to be a ‘snow brother’ and is appreciated for its suitability for driving on snow. In this case, the advertising team should invest more in those snowy regions in the north.


#### Fiat performance related comment visualization:

In [ ]:
fiat_performance_comment = []
for j in range(len(fiat_comment)):
    for i in range(len(fiat_comment[j])):
        if fiat_comment[j][i] in attributes.performance.dropna().tolist():
            fiat_performance_comment.append(list(set(fiat_comment[j])))

In [ ]:
fiat_performance_list = []
for i in range(len(fiat_performance_comment)):
    for j in range(len(fiat_performance_comment[i])):
        fiat_performance_list.append(fiat_performance_comment[i][j])
text = ' '.join(fiat_performance_list).lower()

tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
n_list = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]
n_string=(" ").join(n_list)
adj_string=(" ").join(adj_list)

In [ ]:
wordcloud1 = WordCloud().generate(n_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud1 = WordCloud().generate(adj_string)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

## Task: Discover the most aspirational brand in the discussion

The definition of the aspirational brand found from literature search:

The aspirational brand means which brand:

    1) People want to own, but not already owned for now
    2) And the motivation of wants should be related to the brand's emotional value.
    3) It should be achievable (to delineate the concept from just 'luxurious' brands)

But the third definition can not be described from the current dataset; in field research, the survey method is usually utilized to distinguish an aspirational brand from a luxury brand. (reference: I've Wanted A BMW Since I Was A Kid: An Exploratory Analysis Of The Aspirational Brand, 2015, The Journal of Applied Business Research) Therefore, I decided to focus on finding the set of terms that can serve the definition no.1 and no.2. 


Methodology (Steps)

* to define 'aspirational words' in the given dataset

    * scope to 'adj' words using nltk tagging
    * using VADER, gauge positive score of each word
    * filter only with positivity score is greater than 0.5
    * manually delete irrelevant words from positive word set
    * manually delete functional positivity words from positive word set
        * positive word set consists of 'functional positivity (including 'value for money" and 'emotional positivity' - I decided to keep only 'emotional' part.
    
* With the derived aspirational list of words, calculate lift value for the top 10 brands.
   * only keep rows whose lift value is greater than 1 
* From the lift scores, calculate 'average lift' with aspirational terms per brand.
* The brand shows the highest mean of lifts among aspirational terms is the most aspirational brand.

In [ ]:
wordlist = []
for i in range(len(target_comment)):
    for j in range(len(target_comment[i])):
        wordlist.append(target_comment[i][j])
text = ' '.join(wordlist).lower()

In [ ]:
tokens = nltk.word_tokenize(text)
tags = nltk.pos_tag(tokens)
adj_list = [word for word, pos in tags if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos =='RB' or pos =='RBR'
                                          or pos =='RBS')]

### Define aspiration word set

In [ ]:
#using VADER, gauge positive score of each word 
#and narrow down the words of interest with positivity score is greater than 0.5

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer() 
#sent.polarity_scores('like')

pos_word_dict = {} 
for i in adj_list: #adj_list
    if sent.polarity_scores(i)['pos'] > 0.5:  #filter with positivity score > 0.5
        pos_word_dict[i] = sent.polarity_scores(i)['pos']
        #print(i, sent.polarity_scores(i), sent.polarity_scores(i)['pos'])
    
#pos_word_dict

In [ ]:
pos_list = list(pos_word_dict.keys())

In [ ]:
#eliminate irrelevant words from pos_list
irr_list = ['ha','thanks-', 'well-', 'yay','hug','pray', 'truer','fiestas','gl','fresh', 'bless','smile', 'admit', 'sure', 'highlight','grant','forgiveness', 'agree', 'forgive', 'popular','thank', 'generous', 'clear', 'silly', 'curious', 'certainly', 'welcome', 'yeah', 'peculiar', 'definitely', 'luckily', 'honest', 'nicely', 'certain', 'straight', 'ok', 'honesty', 'create', 'please', 'win', 'grey', 'true', 'honestly', 'convince', 'ready', 'thankfully', 'dear', 'kiss', 'gently', 'yes', 'truly', 'extend', 'strongly', 'hopefully', 'carefully', 'important', 'matter', 'friendly', 'tx', 'tia', 'substantial', 'courtesy', 'clearly', 'respectively', 'greater', 'allow', 'join', 'humorous', 'treat', 'definite', 'funny', 'friend', 'greet', 'optimistically', 'gla', 'agreeable', 'invite', 'lol', 'surprise', 'sigh', 'apologize', 'excuse', 'ly', 'sunny', 'god', 'acceptable', 'okay', 'yep', 'casually', 'ftw', 'alert', 'hilarious', 'accept', 'sincerely', 'importantly', 'respectfully', 'nh', 'laugh', 'goodness', 'reach', 'positively', 'resolve', 'luckier', 'natural', 'fyi', 'thx', 'friends', 'surprisingly', 'truthfully', 'surely', 'truth', 'relieve', 'haha']
pos_list = [item for item in pos_list if item not in irr_list]

#eliminate functional positivity related words
func_list = ['prevent','support','adopt','clean','solve','tolerance','incentive', 'huge', 'helpful', 'easily', 'comfortable','scoop', 'profitable', 'protect', 'defensive', 'careful', 'easier', 'fit', 'smart', 'gt', 'freeway', 'care', 'easy', 'recommend', 'safe', 'casual', 'solid', 'save', 'responsible', 'useful', 'efficient', 'benefit', 'legal', 'stealthily', 'effectively', 'improve', 'reward', 'capable', 'healthy', 'effective', 'stable', 'powerful', 'easiest', 'comprehensive', 'warm', 'help', 'safely', 'faithful', 'incentives', 'painless', 'cleaner', 'mandatory']
pos_list = [item for item in pos_list if item not in func_list]

In [ ]:
aspirations = pos_list

In [ ]:
len(aspirations)

### Calculate lift value for the combination of aspiration terms and the top 10 brands

In [ ]:
#Make combination of Top 10 Brand and aspirations
import itertools
comb = list(itertools.product(top_10,aspirations))

In [ ]:
# Calculate Brand frequency
brand_dict = {}
for (key, value) in brand_frequency.items():
    if key in top_10:
        brand_dict[key] = value

In [ ]:
# Calculate aspirational words' mentioned frequency
asp_frequency = defaultdict(int)
for comment in processed_text: 
    for words in comment: 
        if words in aspirations:
            asp_frequency[words]+=1

#asp_frequency

In [ ]:
# Calculate the #(A,B); pair frequency
pair_frequency = defaultdict(int)
for pair in comb:
    for comm in target_comment_distinct:
        if pair[0] in comm and pair[1] in comm:
            pair_frequency[pair] +=1

In [ ]:
#calculate 'liftA'; lift values for aspirations - brands
liftA = {}
for i in pair_frequency.keys():
    #print(pair_frequency[i], brand_dict[i[0]], asp_frequency[i[1]])
    liftA[i] = i[0], (5000*pair_frequency[i]) / (brand_dict[i[0]]*asp_frequency[i[1]])

In [ ]:
#only keep rows whose lift value > 1 
df_A = pd.DataFrame.from_dict(liftA, orient='index', columns=['brand','lift'])
df_A.sort_values(ascending = True, by = 'lift').head(30)
df_A = df_A[df_A.lift > 1]

### Find out the most aspirational brand - utilizing mean of aspirational lifts per brand

In [ ]:
#Calculate 'average lift' with aspirational terms per brand
df_asp = df_A.groupby('brand').mean().sort_values(ascending = False, by = 'lift')

In [ ]:
#find out the most aspirational brand
print('\n \033[1m' + 'Subaru is the "Most Aspirational" Brand ' + '\033[0m')
print('\n Brands and Lift Values:')
display(df_asp)

From the research, we could prove that the high aspirational orientation was actually realized as the market performance of the brand. From the below chart, we can see that the units sold of Subra in the United States from 2001 to 2019 ( IHS Global Intelligence - Automotive Market Database (released in April, 2019)) soared with 8% of CAGR. Segment-wise, the SUV(Sports Utility Vehicle) models are in orange colours, and the sedan models are in green colours. We can also find out that Subaru’s new car sales are leveraged by SUV models significantly; in 2019, SUV models accounted for more than half of Subaru’s annual sales.

Moreover, connected to the terms such as ‘enjoyable,’ which Subaru showed the maximum lift among brands, we can infer that from 2001 to 2019, the direction of aspirations changed from family sedans to sporty vehicles in the United States passenger car market, as shown in the historical new car sales broken down into segment in the chart. (IHS Global Intelligence - Automotive Market Database (released in April, 2019))


In conclusion, Subaru’s SUV portfolios successfully positioned in growing sporty vehicles demand in the United States, and it led to the high aspirational orientation in consumer’s perception and turned out to increase in sales.